In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("/Users/reem/Desktop/Langchain_tutorials/covered_topics.pdf"),
    PyPDFLoader("/Users/reem/Desktop/Langchain_tutorials/llm.pdf"),
    # PyPDFLoader("/Users/reem/Desktop/Langchain_tutorials/covered topics.pdf")
    # PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture02.pdf"),
    # PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'

In [4]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

/var/folders/wp/g4_x_xzn5dj59z7qh4r7bmr40000gn/T/ipykernel_2850/2897838732.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
/var/folders/wp/g4_x_xzn5dj59z7qh4r7bmr40000gn/T/ipykernel_2850/2897838732.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [5]:
print(vectordb._collection.count())

544


In [6]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [7]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [8]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [9]:
smalldb.similarity_search(question, k=2)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

In [10]:
#MMR  IT CAPTURES DIVERSITY
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

In [11]:
q= "what are the projects? "

In [12]:
print(docs[0].page_content)

Covered Topics 
• OpenCV Multimedia Processing: 
- Image Filtering Basics 
- Morphological Image processing 
- BITWISWE  
- Transformation Functions 
- Feature Extraction 
- Operations 
- Blurring Techniques 
- Color Spaces 
- Contour Detection 
- Edge Detection 
- Masking 
- Threshold Techniques 
- Face Detection Using haar cascade 
Reference: 
https://drive.google.com/file/d/14BsnVqC2c6NNsV4MrAg79ltP_h6Fzd5e/vi
ew?usp=share_link 
 
• Object Detection And Tracking With Ultralytics YOLO: 
- Model’s Versions And Capabilities Exploration 
- Bounding Boxes Basics(IOU, mAP, Nms, Confidence Threshold) 
- Understanding Model’s Functionalities


In [13]:
print(docs[2].page_content)

3 
Courses 
• Neural Networks And Deep learning with Andrew Ng 
• Prompt Engineering With Llama 2&3


In [14]:
question = "What are projects in the third document?"

In [15]:
docs = vectordb.similarity_search(question,k=5)

In [16]:
for doc in docs:
    print(doc.metadata)          #it got it from not only the specifies document but other docs as well

{'author': 'Reem Alamoudi', 'creationdate': '2025-02-26T10:46:13+00:00', 'creator': 'Microsoft Word', 'moddate': '2025-02-26T10:46:13+00:00', 'page': 3, 'page_label': '4', 'producer': 'PyPDF', 'source': '/Users/reem/Desktop/Langchain_tutorials/covered topics.pdf', 'total_pages': 6}
{'author': 'Reem Alamoudi', 'creationdate': '2025-02-26T10:46:13+00:00', 'creator': 'Microsoft Word', 'moddate': '2025-02-26T10:46:13+00:00', 'page': 4, 'page_label': '5', 'producer': 'PyPDF', 'source': '/Users/reem/Desktop/Langchain_tutorials/covered topics.pdf', 'total_pages': 6}
{'author': '', 'creationdate': '2025-01-16T20:13:48-05:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2025-01-16T20:13:48-05:00', 'page': 117, 'page_label': '111', 'producer': 'GPL Ghostscript 10.01.2', 'source': '/Users/reem/Desktop/Langchain_tutorials/llm.pdf', 'subject': '', 'title': '', 'total_pages': 231}
{'author': '', 'creationdate': '2025-01-16T20:13:48-05:00', 'creator': 'LaTeX with hyperref', 'keyword

In [17]:
print(docs[4].page_content) #it is not picking up that we only want info from the third doc  , we can either add a filter or use SelfQuery

3 
Courses 
• Neural Networks And Deep learning with Andrew Ng 
• Prompt Engineering With Llama 2&3


In [18]:
#manually specifiying :
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"docs/cs229_lectures/MachineLearning-Lecture03.pdf"}
)

In [19]:
for doc in docs:
    print(doc.metadata)    

## SelfQuery

### Addressing Specificity: working with metadata using self-query retriever

But we have an interesting challenge: we often want to infer the metadata from the query itself.

To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:
 
1. The `query` string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

In [20]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo   #where we can specify different fields in the metadata

In [21]:
# this will be pased to the language model, so it is important to nake it as descriptive as possible
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `/Users/reem/Desktop/Langchain_tutorials/covered topics.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [22]:
import lark
document_content_description = "Covered topics"
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

/var/folders/wp/g4_x_xzn5dj59z7qh4r7bmr40000gn/T/ipykernel_2850/90427950.py:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)


In [25]:
question = "What are projects in the  document?"

In [26]:
docs = retriever.get_relevant_documents(question)

In [27]:
for d in docs:
    print(d.metadata)

{'author': '', 'creationdate': '2025-01-16T20:13:48-05:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2025-01-16T20:13:48-05:00', 'page': 1, 'page_label': 'i', 'producer': 'GPL Ghostscript 10.01.2', 'source': '/Users/reem/Desktop/Langchain_tutorials/llm.pdf', 'subject': '', 'title': '', 'total_pages': 231}
{'author': '', 'creationdate': '2025-01-16T20:13:48-05:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2025-01-16T20:13:48-05:00', 'page': 126, 'page_label': '120', 'producer': 'GPL Ghostscript 10.01.2', 'source': '/Users/reem/Desktop/Langchain_tutorials/llm.pdf', 'subject': '', 'title': '', 'total_pages': 231}
{'author': '', 'creationdate': '2025-01-16T20:13:48-05:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2025-01-16T20:13:48-05:00', 'page': 117, 'page_label': '111', 'producer': 'GPL Ghostscript 10.01.2', 'source': '/Users/reem/Desktop/Langchain_tutorials/llm.pdf', 'subject': '', 'title': '', 'total_pages': 231}
{'author

### Additional tricks: compression

Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text. 

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this. 

In [26]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [27]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [28]:
# Wrap our vectorstore
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")
compressor = LLMChainExtractor.from_llm(llm)

In [29]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [34]:
question = "what did they say about projects?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

- Defining Project Pipeline And Requirements 
- Custom Dataset Processing (Collection, Labelling, Annotation) 
- Fine Tuning Pre-Trained Model (YOLOv11) 
- Testing The Model Then Deploying it
----------------------------------------------------------------------------------------------------
Document 2:

T axi Driver: Deﬁnitely check out the botanical gardens after the museum.
They’re nearby and quite beautiful.
T ourist: Great, thanks for the tip!
T axi Driver: No problem. Enjoy your visit!
----------------------------------------------------------------------------------------------------
Document 3:

Another application of text completion is generating text b ased on user requirements. The
requirements can include style, tone, length, and any speci ﬁc content that the text should contain.
For example, users may need a formal report, a creative story , or a piece of programming code.


In [30]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [32]:
question = "what did they say about projects?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

- Defining Project Pipeline And Requirements 
- Custom Dataset Processing (Collection, Labelling, Annotation) 
- Fine Tuning Pre-Trained Model (YOLOv11) 
- Testing The Model Then Deploying it
----------------------------------------------------------------------------------------------------
Document 2:

Another application of text completion is generating text b ased on user requirements. The
requirements can include style, tone, length, and any speci ﬁc content that the text should contain.
For example, users may need a formal report, a creative story , or a piece of programming code.
----------------------------------------------------------------------------------------------------
Document 3:

involving multiple prompts
----------------------------------------------------------------------------------------------------
Document 4:

• Language Modeling . T ypically , this approach refers to an auto-regressive gen eration pro-
cedure of sequences. At one time, it predic

## Other types of retrieval

It's worth noting that vectordb as not the only kind of tool to retrieve documents. 

The `LangChain` retriever abstraction includes other ways to retrieve documents, such as TF-IDF or SVM.

In [33]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [34]:
# Load PDF
loader = PyPDFLoader("/Users/reem/Desktop/Langchain_tutorials/covered topics.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)


In [36]:
# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [ ]:
question = "What are major topics for this class?"
docs_svm=svm_retriever.get_relevant_documents(question)
docs_svm[0]

In [ ]:
question = "what did they say about matlab?"
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]